# AI Grader
Use LLM to grade student answer

In [ ]:
%pip install langchain openai python-dotenv pandas openpyxl tqdm

In [ ]:
import dotenv
# Load .env file
dotenv.load_dotenv('.env')

In [ ]:
deployment_name = "gpt-35-turbo"
model_name = "gpt-35-turbo"
base_folder = "data/ITE3101_practical_tests/ite-3101-practical-test-ab-submissions/"
answer_excel = "student_answer.xlsx"
answer_excel_path = base_folder + answer_excel
standard_answer_path = base_folder + "standard_answer_ab.xlsx"

In [ ]:
import pandas as pd

student_answer_df = pd.read_excel(answer_excel_path)
student_answer_df.head()

In [ ]:
standard_answer = pd.read_excel(standard_answer_path)
standard_answer.head()

In [ ]:
standard_answer_dict = standard_answer.set_index(
    'Question Name').to_dict(orient='index')
standard_answer_dict

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
import json
import langchain
langchain.debug = False


llm = AzureChatOpenAI(
    deployment_name=deployment_name,
    model_name=model_name,
    temperature=0.1
)


def score_answer(instruction, starter, answer, mark, student_answer, student_commit):
    template = "You are a Python programming instructor who grades student Python exercises."
    # load grader_prompt.txt
    with open("grader_prompt.txt") as f:
        grader_prompt = f.read()

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("human", grader_prompt),
    ])
    # print(chat_prompt)
    chain = chat_prompt | llm
    data = {"instruction": instruction,
            "starter": starter,
            "answer": answer,
            "mark": mark,
            "student_answer": student_answer,
            "student_commit": student_commit}

    r = chain.invoke(data).content
    return r

In [ ]:
import json
from tqdm import tqdm

rows = []
for index, row in tqdm(student_answer_df.iterrows(), total=len(student_answer_df)):
    for key, value in standard_answer_dict.items():
        question = key
        instruction = value["Instruction"]
        starter = value["Starter"]
        answer = value["Answer"]
        mark = value["Mark"]
        student_answer = row[key + " Content"]
        student_commit = row[key + " Commit"]

        for _ in range(3):  # Retry 3 times
            try:
                result = score_answer(
                    instruction, starter, answer, mark, student_answer, student_commit)
                result = json.loads(result)  # Parse the JSON response
                break  # Break the loop if successful
            except json.JSONDecodeError:
                continue  # Retry if JSON decoding error occurs

        row[key + " Score"] = result["score"]
        row[key + " Comments"] = result["comments"]
        row[key + " Calculation"] = result["calculation"]
        row[key + " Confident"] = result["confident"]

    rows.append(row)

In [ ]:
scored_df = pd.DataFrame(rows)
scored_df.head()

In [ ]:
scored_df[scored_df.filter(like='Score').columns] = scored_df.filter(
    like='Score').astype(int)
scored_df[scored_df.filter(like='Confident').columns] = scored_df.filter(
    like='Confident').astype(int)
scored_df[scored_df.filter(like='Commit').columns] = scored_df.filter(
    like='Commit').astype(int)
scored_df['total_score'] = scored_df.filter(like='Score').sum(axis=1)
scored_df.head()

In [ ]:
score_columns = scored_df.filter(like='Score').columns
scored_df = scored_df[[
    col for col in scored_df.columns if col not in score_columns] + list(score_columns)]
scored_df.head()

In [ ]:
excel_file_path = os.path.join(base_folder, 'student_score.xlsx')
scored_df.to_excel(excel_file_path, index=False)